## Import Packages

In [1]:
import os
import azureml
import shutil

from azureml.core.authentication import AzureCliAuthentication
from azureml.core import Workspace, Datastore, Experiment, Environment, ScriptRunConfig

from azureml.core.compute import ComputeTarget, AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core.dataset import Dataset
from azureml.core.resource_configuration import ResourceConfiguration

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.41.0


## Connect to the Workspace

In [2]:
cli_auth = AzureCliAuthentication()

ws = Workspace(
    subscription_id="",
    resource_group="",
    workspace_name="",
    auth=cli_auth,
)

print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace ml-poc-ws at location westeurope


In [3]:
cluster_name = "cc-shared-nc4-T4-v3"
compute_target = None

try:
    # Check for existing compute target
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
        compute_target.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
else:
    print("Compute Target:", compute_target.name)

Found existing cluster, use it.
Compute Target: cc-shared-nc4-T4-v3


In [4]:
# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:", ds.name)

# Get access to Flowers dataset
flowers_ds = Dataset.get_by_name(ws, name='flowers-dataset')

Datastore: workspaceblobstore


In [5]:
# Connect to the experiment
exp = Experiment(workspace=ws, name='flowers-clf-experiment')
print("Experiment:", exp.name)

Experiment: flowers-clf-experiment


## Train model & Publish training pipeline

### Create the environment

In [6]:
env_name = 'flowers-train-pytorch-1-10-ubuntu18-py38-cuda11-gpu'
pytorch_env = Environment.get(workspace=ws, name=env_name)

### Create the training pipeline

In [7]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

pipeline_run_config = RunConfiguration()
pipeline_run_config.target = compute_target
pipeline_run_config.environment = pytorch_env

args = [
    '--data-folder', flowers_ds.as_named_input('flowers').as_mount(),
]

project_folder = "./"

train_step = PythonScriptStep(
    name="Train flowers model",
    source_directory=project_folder,
    script_name="train.py",
    arguments=args,
    compute_target=compute_target,
    runconfig=pipeline_run_config,
    allow_reuse=True
)

print('Steps defined')

Steps defined


In [8]:
train_pipeline_steps = [train_step]
train_pipeline = Pipeline(workspace=ws, steps=train_pipeline_steps)

### Train the model

In [9]:
# args = [
#     '--data-folder', flowers_ds.as_named_input('flowers').as_mount(),
# ]

# project_folder = "./"

# config = ScriptRunConfig(
#     source_directory=project_folder, 
#     script='train.py', 
#     compute_target=compute_target,
#     environment=pytorch_env,
#     arguments=args,
# )

In [10]:
pipeline_run = exp.submit(train_pipeline, regenerate_outputs=True)
RunDetails(pipeline_run).show()
# pipeline_run.wait_for_completion(show_output=True)

Created step Train flowers model [39d379a1][cb1f0157-cd3f-4114-824f-46a27d6d5ace], (This step will run and generate new outputs)
Submitted PipelineRun ce7edf44-5d25-4020-a4b9-9932cf82668e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ce7edf44-5d25-4020-a4b9-9932cf82668e?wsid=/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourcegroups/ml-poc/workspaces/ml-poc-ws&tid=9064dc66-330c-438e-aee3-1c9e605cc16d


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
flowers-clf-experiment,ce7edf44-5d25-4020-a4b9-9932cf82668e,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Show info about metrics and models

In [16]:
from azureml.core import Model

for model in Model.list(ws):
    if not model.name.startswith('Flowers-clf-PyTorch'):
        continue
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Flowers-clf-PyTorch version: 3
	 Training context : Pipeline


Flowers-clf-PyTorch version: 2


Flowers-clf-PyTorch version: 1




### Register the model in Azure ML

In [17]:
# model = pipeline_run.register_model(
#     model_name='Flowers-clf-PyTorch',
#     model_path='outputs/models/FlowersConvModel_model_best.pth',
#     description="Flowers PyTorch Classifier",
#     resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=2)
# )

# print("Model '{}' version {} registered ".format(model.name, model.version))

### Publish training pipeline

In [18]:
published_pipeline = pipeline_run.publish_pipeline(
    name='flowers-train-pipeline', description='Train flowers model', version='1.0'
)
published_pipeline

Name,Id,Status,Endpoint
flowers-train-pipeline,c0fdd530-63af-4656-9dfc-8f769bfbec4a,Active,REST Endpoint


In [19]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourceGroups/ml-poc/providers/Microsoft.MachineLearningServices/workspaces/ml-poc-ws/PipelineRuns/PipelineSubmit/c0fdd530-63af-4656-9dfc-8f769bfbec4a


## Publish inference pipeline

### Create an environment

In [30]:
env_name = 'flowers-infer-pytorch-1-10-ubuntu18-py38-cuda11-gpu'
pytorch_env = Environment.get(workspace=ws, name=env_name)

### Create a compute target

In [32]:
cluster_name = "cc-shared-nc4-T4-v3"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


### Create a infer pipeline

In [33]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig

In [34]:
output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory='./',
    entry_script="infer_batch.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=pytorch_env,
    compute_target=inference_cluster,
    node_count=1
)

parallelrun_step = ParallelRunStep(
    name='flowers-batch-score',
    parallel_run_config=parallel_run_config,
    inputs=[flowers_ds.as_named_input('flowers').as_mount()],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


In [35]:
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'flowers-infer-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step flowers-batch-score [0d4b2959][85dfb5f8-2d2a-4892-8916-c325d4d12438], (This step will run and generate new outputs)
Submitted PipelineRun aa2eb0e7-a05c-4d39-8564-49a5985a2f4e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/aa2eb0e7-a05c-4d39-8564-49a5985a2f4e?wsid=/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourcegroups/ml-poc/workspaces/ml-poc-ws&tid=9064dc66-330c-438e-aee3-1c9e605cc16d
PipelineRunId: aa2eb0e7-a05c-4d39-8564-49a5985a2f4e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/aa2eb0e7-a05c-4d39-8564-49a5985a2f4e?wsid=/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourcegroups/ml-poc/workspaces/ml-poc-ws&tid=9064dc66-330c-438e-aee3-1c9e605cc16d
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: dd187426-e4bc-408a-af1d-72d71612c29b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/dd187426-e4bc-408a-af1d-72d71612c29b?wsid=/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resour

'Finished'

In [36]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('../output/predictions', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='../output/predictions')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('../output/predictions'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,100080576_f52e8ee070_n.jpg,[0]
1,10140303196_b88d3d6cec.jpg,[0]
2,10172379554_b296050f82_n.jpg,[0]
3,10172567486_2748826a8b.jpg,[0]
4,102841525_bd6628ae3c.jpg,[0]
5,10437754174_22ec990b77_m.jpg,[0]
6,10437770546_8bb6f7bdd3_m.jpg,[0]
7,10437929963_bc13eebe0c.jpg,[0]
8,10466558316_a7198b87e2.jpg,[0]
9,10555749515_13a12a026e.jpg,[0]


### Deploy Pipeline

In [37]:
published_pipeline = pipeline_run.publish_pipeline(
    name='flowers-infer-batch-pipeline', description='Batch scoring of flowers data', version='1.0'
)
published_pipeline

Name,Id,Status,Endpoint
flowers-infer-batch-pipeline,caab782f-f3cb-458b-be04-5d53612f0e2b,Active,REST Endpoint


In [38]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourceGroups/ml-poc/providers/Microsoft.MachineLearningServices/workspaces/ml-poc-ws/PipelineRuns/PipelineSubmit/caab782f-f3cb-458b-be04-5d53612f0e2b


In [39]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Authentication header ready.


In [41]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(
    rest_endpoint, 
    headers=auth_header, 
    json={"ExperimentName": "flowers-infer-batch"}
)
run_id = response.json()["Id"]
run_id

'c638a8e9-93b5-405f-a557-6296b3b6fdad'

In [42]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments['flowers-infer-batch'], run_id)

# Block until the run completes
published_pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: c638a8e9-93b5-405f-a557-6296b3b6fdad
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c638a8e9-93b5-405f-a557-6296b3b6fdad?wsid=/subscriptions/d752c461-4a37-4d63-a15d-ec58b07063cb/resourcegroups/ml-poc/workspaces/ml-poc-ws&tid=9064dc66-330c-438e-aee3-1c9e605cc16d

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': 'c638a8e9-93b5-405f-a557-6296b3b6fdad', 'status': 'Completed', 'startTimeUtc': '2022-05-26T15:30:27.05218Z', 'endTimeUtc': '2022-05-26T15:30:28.395376Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'Unavailable', 'runType': 'HTTP', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.pipelineid': 'caab782f-f3cb-458b-be04-5d53612f0e2b', 'azureml.pipelineComponent': 'pipelinerun'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://mlpocwsstorage21a89edcd9.blob.core.windows.net/azureml/ExperimentRun/dcid.c638a8

'Finished'

In [43]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('../output/predictions', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='../output/predictions')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('../output/predictions'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,100080576_f52e8ee070_n.jpg,[0]
1,10140303196_b88d3d6cec.jpg,[0]
2,10172379554_b296050f82_n.jpg,[0]
3,10172567486_2748826a8b.jpg,[0]
4,102841525_bd6628ae3c.jpg,[0]
5,10437754174_22ec990b77_m.jpg,[0]
6,10437770546_8bb6f7bdd3_m.jpg,[0]
7,10437929963_bc13eebe0c.jpg,[0]
8,10466558316_a7198b87e2.jpg,[0]
9,10555749515_13a12a026e.jpg,[0]
